In [ ]:
### START FROM HERE
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# General installation of packages

!pip install cobra
import cobra
from cobra import Model, Reaction, Metabolite



In [3]:
# Import model
model=cobra.io.read_sbml_model('/content/drive/MyDrive/REPLAY - A2C/Altri lavori paralleli/_TALTECH/Modeling/Rt_IFO0880.xml') # Vittorio
#model=cobra.io.read_sbml_model('/content/drive/MyDrive/TalTech/modelling/Rt_IFO0880.xml') # Alina
#model #to check it worked


#!pip install escher
#import escher
#from escher import Builder

In [ ]:
# Putative ethylene glycol pathway:

# ethylene glycol oxidase: ethylene glycol DH
# GRE2: RHTO_02276 - 12151 (LALDO3), RHTO_04342 - 11160 (LALDO3), RHTO_03406 - 10497 (LALDO3), RHTO_04716 - 12315 (), RHTO_00738 - 11786 ()
# ALD1/ALD6:
# YLL056C: RHTO_04070 - 9601 (), RHTO_04071 - 9602 ()

# glycolaldehyde oxidase: glycolaldehyde DH
# ALDH: RHTO_05835 - 12042 (GCALDD)
# ALD3
# ALD4
# ALD5

# glyoxylate reductase:
# GOR1: RHTO_05831 - 12051 (GLYCLTDxm)

In [4]:
### Add EG as a metabolite

# 1. Add EG to the cytosol and the outside of the cell
model.add_metabolites([
            Metabolite(
            'eg_c',
            formula='C2H6O2',
            name='eg_c',
            compartment='c'
            ),
            Metabolite(
            'eg_e',
            formula='C2H6O2',
            name='eg_e',
            compartment='e'   
            ),
])


# 2. Create transport reaction to allow EG to enter the cell (from e to c)
eg_e=model.metabolites.get_by_id('eg_e')
eg_c=model.metabolites.get_by_id('eg_c')

EGt = Reaction('T_eg')
EGt.name = 'EG transport'
EGt.subsystem = 'EG Oxydation Pathway'
EGt.lower_bound = -999999
EGt.upper_bound = 999999

EGt.add_metabolites({
                eg_e: -1.0,
                eg_c: 1.0
})

model.add_reactions([EGt])
#model.reactions.get_by_id("T_eg") # check


# 3. Create exchange reaction to allow EG to enter the cell
EX_eg_e = Reaction('EX_eg_e')
EX_eg_e.name = 'EG exchange'
EX_eg_e.subsystem = 'EG Oxydation Pathway'
EX_eg_e.lower_bound = -999999
EX_eg_e.upper_bound = 999999

EX_eg_e.add_metabolites({
                eg_e: -1.0,
})

model.add_reactions([EX_eg_e])
#model.reactions.get_by_id("EX_eg_e") # check

# set bounds for EG exchange
#model.add_boundary(model.metabolites.get_by_id('eg_e'), type="exchange")

# check if it has been created (and the name); rxn are added at the bottom
#model.exchanges


In [5]:
### Create the new reactions for EG consumption


## 1. EG oxidation to GAH (NAD+) - homologue to YLL056C (Sc)
eg_ox_nad = Reaction('EG_ox_nad')
eg_ox_nad.name = 'EG - NAD-dependent alcohol dehydrogenase'
eg_ox_nad.subsystem = 'EG Oxydation Pathway'
eg_ox_nad.lower_bound = -999999
eg_ox_nad.upper_bound = 999999

## 2. EG oxidation to GAH (NADP+) - homologue to GRE2 (Sc)
eg_ox_nadp = Reaction('EG_ox_nadp')
eg_ox_nadp.name = 'EG - NADP-dependent alcohol dehydrogenase'
eg_ox_nadp.subsystem = 'EG Oxydation Pathway'
eg_ox_nadp.lower_bound = -999999
eg_ox_nadp.upper_bound = 999999

## 3. Get the metabolites involved in the reactions
eg_c=model.metabolites.get_by_id("eg_c")        # get EG
nad_c=model.metabolites.get_by_id("nad_c")      # get NAD+
nadh_c=model.metabolites.get_by_id("nadh_c")    # get NADH
h_c=model.metabolites.get_by_id("h_c")          # get a proton
nadp_c=model.metabolites.get_by_id("nadp_c")     # get NADP+
nadph_c=model.metabolites.get_by_id("nadph_c")   # get NADPH
gah_c=model.metabolites.get_by_id('gcald_c')    # get GAH

## 4. Add metabolites to reactions and change bounds

# 4a. NAD+ dependent
eg_ox_nad.add_metabolites({
              eg_c: -1.0,
              nad_c: -1.0,
              gah_c: 1.0,
              nadh_c: 1.0,
              h_c: 1.0,
})

#eg_ox_nad.reaction  # to check

# change bounds
eg_ox_nad.lower_bound = 0



# 4b. NADP+ dependent
eg_ox_nadp.add_metabolites({
              eg_c: -1.0,
              nadp_c: -1.0,
              gah_c: 1.0,
              nadph_c: 1.0,
              h_c: 1.0,
})

#eg_ox_nadp.reaction  # to check

# change bounds
eg_ox_nadp.lower_bound = 0


## 5. Add reactions to model
model.add_reactions([eg_ox_nad])
model.add_reactions([eg_ox_nadp])


In [6]:
### Change bounds for the reactions GA <-> GOX

# 1. Glyoxylate reductase (NAD), mitochondrial
ga_ox_mit=model.reactions.get_by_id('GLYCLTDxm')
ga_ox_mit.bounds = (-1000, 1000)
#ga_ox_mit # to check

# 2. Glyoxylate reductase (NADP), cytosol
ga_ox_c=model.reactions.get_by_id('GLYCLTDy')
ga_ox_c.bounds = (-1000, 1000)
#ga_ox_c # to check


# Remove oxidation of glycolate to glyoxylate with O2
GLYCTO1p = model.reactions.get_by_id("GLYCTO1p")
model.remove_reactions([GLYCTO1p])


In [7]:
# MODEL CURATION: DEFAULT MODEL

# Transfer of AcCoA between compartments

# Read all involved metabolites
acrn_x=model.metabolites.get_by_id("acrn_x")
acrn_c=model.metabolites.get_by_id("acrn_c")
acrn_m=model.metabolites.get_by_id("acrn_m")
crn_x=model.metabolites.get_by_id("crn_x")
crn_c=model.metabolites.get_by_id("crn_c")
crn_m=model.metabolites.get_by_id("crn_m")
coa_c=model.metabolites.get_by_id("coa_c")
coa_m=model.metabolites.get_by_id("coa_m")
accoa_c=model.metabolites.get_by_id("accoa_c")

# deacetylation reaction in cytosol
CSNATr = Reaction("CSNATr")
CSNATr.name = 'Carnitine O acetyltransferase'
CSNATr.lower_bound = -1000    #reversible
CSNATr.upper_bound = 1000

CSNATr.add_metabolites({
              acrn_c: -1.0,
              coa_c: -1.0,
              accoa_c: 1.0,
              crn_c: 1.0,
              })

#Exchange of TCA intermediates between peroxisome and cytosol


# CITtcp - Citrate/isocitrate antiport into peroxisome

cit_c=model.metabolites.get_by_id("cit_c")
cit_x=model.metabolites.get_by_id("cit_x")
icit_c=model.metabolites.get_by_id("icit_c")
icit_x=model.metabolites.get_by_id("icit_x")

CITtcp = Reaction("CITtcp")
CITtcp.name = 'Citrate/isocitrate antiport into peroxisome'
CITtcp.lower_bound = -1000    #reversible
CITtcp.upper_bound = 1000

CITtcp.add_metabolites({
              cit_c: -1.0,
              icit_x: -1.0,
              icit_c: 1.0,
              cit_x: 1.0,
              })


# GLYCLTDyp - Glyoxylate reductase (NADP), peroxisomal: change bounds to make it reversible
GLYCLTDyp=model.reactions.get_by_id('GLYCLTDyp')
GLYCLTDyp.bounds = (-1000, 1000)


# GLYCLTtp - GA transport between cytosol and peroxisome: change bounds to make it reversible
GLYCLTtp=model.reactions.get_by_id('GLYCLTtp')
GLYCLTtp.bounds = (-1000, 1000)


# ACONT [cytosolic] (aconitase) is missing from the model

ACONTc = Reaction("ACONTc")
ACONTc.name = 'cytosolic aconitase'
ACONTc.lower_bound = -1000    #reversible
ACONTc.upper_bound = 1000

ACONTc.add_metabolites({
              cit_c: -1.0,
              icit_c: 1.0,
              })

# ICLc - cytosolic Isocitrate lyase (gene ICL2)
icit_m=model.metabolites.get_by_id("icit_m")
glx_c=model.metabolites.get_by_id("glx_c")
glx_m=model.metabolites.get_by_id("glx_m")
succ_c=model.metabolites.get_by_id("succ_c")
succ_m=model.metabolites.get_by_id("succ_m")

  # cytosolic
ICLc = Reaction("ICLc")
ICLc.name = 'cytosolic Isocitrate lyase'
ICLc.lower_bound = -1000    #reversible
ICLc.upper_bound = 1000

ICLc.add_metabolites({
              icit_c: -1.0,
              glx_c: 1.0,
              succ_c: 1.0
              })

# MALS (cytosolic malate synthase)
mal__L_c=model.metabolites.get_by_id("mal__L_c")
h2o_c=model.metabolites.get_by_id("h2o_c")

MALS = Reaction("MALS")
MALS.name = ' Isocitrate lyase'
MALS.lower_bound = -1000    #reversible
MALS.upper_bound = 1000

MALS.add_metabolites({
              accoa_c: -1.0,
              glx_c: -1.0,
              h2o_c: -1.0,
              coa_c: 1.0,
              mal__L_c: 1.0,
              h_c: 1.0
              })

# MDHc - cytosolic malate dehydrogenase
oaa_c=model.metabolites.get_by_id("oaa_c")

MDHc = Reaction("MDHc")
MDHc.name = 'cytosolic malate dehydrogenase'
MDHc.lower_bound = -1000    #reversible
MDHc.upper_bound = 1000

MDHc.add_metabolites({
              mal__L_c: -1.0,
              nad_c: -1.0,
              oaa_c: 1.0,
              nadh_c: 1.0,
              h_c: 1.0
              })

# FUMc - cytosolic fumarase
fum_c=model.metabolites.get_by_id("fum_c")

FUMc = Reaction("FUMc")
FUMc.name = 'cytosolic fumarase'
FUMc.lower_bound = -1000    #reversible
FUMc.upper_bound = 1000

FUMc.add_metabolites({
              fum_c: -1.0,
              h2o_c: -1.0,
              mal__L_c: 1.0
              })

# IDP1c - Cytosolic isocitrate dehydrogenase NADP+-dependent
akg_c=model.metabolites.get_by_id("akg_c")
co2_c=model.metabolites.get_by_id("co2_c")

IDP1c = Reaction("IDP1c")
IDP1c.name = 'Cytosolic isocitrate dehydrogenase NADP+-dependent'
IDP1c.lower_bound = -1000    #reversible
IDP1c.upper_bound = 1000

IDP1c.add_metabolites({
              icit_c: -1.0,
              nadp_c: -1.0,
              akg_c: 1.0,
              co2_c: 1.0,
              nadph_c: 1.0,
              h_c: -1.0
              })


model.add_reactions([CSNATr,
                    ACONTc,
                    ICLc,
                    MALS,
                    MDHc,
                    FUMc,
                    IDP1c])

In [23]:
# ALTERNATIVE MODEL alpha:

# Transport of Acetyl-CoA between compartments (peroxisome-cytosol, cytosol-mitochondria) is mediated by carnitine. The reactions are present in the model but they do not keep into consideration that (acetyl-)carnitine is regenerated in the compartments.

# create ACRNtp ("Acetylcarnitine transport to/from peroxisome" should be changed to allow consumption of cytosolic carnitine and regeneration of peroxisomal carnitine
ACRNtp = model.reactions.get_by_id("ACRNtp")
ACRNtp.add_metabolites({
              crn_c: -1.0,
              crn_x: 1.0,
              })

# create ACRNtm ("Acetylcarnitine transport to/from mitochondria")
ACRNtm = model.reactions.get_by_id("ACRNtm")
ACRNtm.add_metabolites({
              crn_c: 1.0,
              crn_m: -1.0,
              })


In [28]:
 #ALTERNATIVE MODEL beta:

# Transport of Acetyl-CoA between compartments (peroxisome-cytosol, cytosol-mitochondria) is mediated by carnitine. The reactions are present in the model but they do not keep into consideration that (acetyl-)carnitine is regenerated in the compartments.

# create ACRNtp ("Acetylcarnitine transport to/from peroxisome" should be changed to allow consumption of cytosolic carnitine and regeneration of peroxisomal carnitine
ACRNtp = model.reactions.get_by_id("ACRNtp")
ACRNtp.add_metabolites({
              crn_c: -1.0,
              crn_x: 1.0,
              })

# create ACRNtm ("Acetylcarnitine transport to/from mitochondria")
ACRNtm = model.reactions.get_by_id("ACRNtm")
ACRNtm.add_metabolites({
              crn_c: 1.0,
              crn_m: -1.0,
              })

 # ICLm (mitochondrial, gene ICL1)
ICLm = Reaction("ICLm")
ICLm.name = 'mitochondrial Isocitrate lyase'
ICLm.lower_bound = -1000    #reversible
ICLm.upper_bound = 1000

ICLm.add_metabolites({
              icit_m: -1.0,
              glx_m: 1.0,
              succ_m: 1.0
              })

model.add_reactions([ICLm])

In [10]:
# Save model:
# json: escher maps
# xml
# mat

### CHECK THE NAME OF THE FILES EVERY TIME!!! ###
cobra.io.json.save_json_model(model,'/content/drive/MyDrive/REPLAY - A2C/Altri lavori paralleli/_TALTECH/Modeling/_Model curation/EG only/Rt_IFO0880_EG_curated_v4.json')
cobra.io.save_matlab_model(model,'/content/drive/MyDrive/REPLAY - A2C/Altri lavori paralleli/_TALTECH/Modeling/_Model curation/EG only/Rt_IFO0880_EG_curated_v4.mat')
cobra.io.write_sbml_model(model,'/content/drive/MyDrive/REPLAY - A2C/Altri lavori paralleli/_TALTECH/Modeling/_Model curation/EG only/Rt_IFO0880_EG_curated_v4.xml')
#cobra.io.json.save_json_model(model,'/content/drive/MyDrive/TalTech/modelling/Rt_IFO0880_EG_curated_v04.json') # Alina # both cofactors
#cobra.io.save_matlab_model(model,'/content/drive/MyDrive/TalTech/modelling/Rt_IFO0880_EG_curated_v04.mat') # Alina
#cobra.io.write_sbml_model(model,'/content/drive/MyDrive/TalTech/modelling/Rt_IFO0880_EG_curated_v04.xml') # Alina


In [ ]:
# We will run two types of simulations: EG and glucose co-consumption (1st) and EG consumption alone (2nd)
### Setting the parameters for the simulation (e.g. medium)

#1st: Default bound is glucose uptake = 1.0
# set glucose uptake
medium = model.medium
medium["EX_glc__D_e"] = 0.1
model.medium = medium

# set EG uptake to 1
medium = model.medium
medium["EX_eg_e"] = 1.0
model.medium = medium

### Allow glycolate accumulation
# constrain glycolate
EX_glyclt_e=model.reactions.get_by_id('EX_glyclt_e')
EX_glyclt_e.upper_bound=0.90
EX_glyclt_e.lower_bound=0.75
EX_glyclt_e

model.medium    # Bounds after setting up

In [ ]:
#2nd:

# set glucose uptake to 0
medium = model.medium
medium["EX_glc__D_e"] = 0.0
model.medium = medium

# set EG uptake to 1
medium = model.medium
medium["EX_eg_e"] = 1.0
model.medium = medium

### Allow glycolate accumulation
# constrain glycolate
EX_glyclt_e=model.reactions.get_by_id('EX_glyclt_e')
EX_glyclt_e.upper_bound=0.90
EX_glyclt_e.lower_bound=0.75
EX_glyclt_e

model.medium    # Bounds after setting up

In [ ]:
# Run flux balance analysis, FBA (optimization function):

solution = model.optimize()
print(solution.objective_value)
print(solution.fluxes)

#If necessary:
# check objective
#print(model.objective.expression)
#print(model.objective.direction)

# if necessary to change the objective:
#model.objective = "ATPM" # or any other reaction

#If necessary:
# Run loopless FBA (parsimonious FBA, pFBA):
#pfba_solution = cobra.flux_analysis.pfba(model)
#pfba_solution

In [ ]:
# Display fluxes: summary

model.summary()

In [32]:
# Export data (for Escher):

import pandas as pd
data = solution.fluxes
#data = pfba_solution.fluxes
data.to_csv('/content/drive/MyDrive/REPLAY - A2C/Altri lavori paralleli/_TALTECH/Modeling/_Model curation/EG only/Rt_IFO0880_EG_curated_v4_fluxes.csv')
#data.to_csv('/content/drive/MyDrive/TalTech/modelling/fluxes_curated_beta_v04.csv') # Alina


In [ ]:
# Other code:

model.metabolites.nadh_x.summary()

In [ ]:
model.metabolites.nadh_m.summary()

In [ ]:
model.metabolites.nadh_c.summary()

In [ ]:
model.metabolites.nadph_x.summary()


In [ ]:
model.metabolites.nadph_m.summary()


In [ ]:
model.metabolites.nadph_c.summary()


In [ ]:
# TODO:
# check cofactors for EG uptake
# save ModelSummary
# save metabolitesSummary (NADP, NAD)